In [ ]:
# !pip3 install git+https://github.com/openai/whisper.git
# !pip3 install ffmpeg

In [28]:
import os
import re
import whisper
import pandas as pd
# import ffmpeg

model = whisper.load_model("tiny")

def transcribe_Whisper_tiny(filename):
    result = model.transcribe(filename, language='zh')
    return result["text"]

df_whisper_tiny = pd.DataFrame({
    'Filename': [],
    'Start_time': [],
    'End_time': [],
    'Whisper_tiny': []
})

In [29]:
for i in range(1, 71):
    number = str(i).zfill(2)
    input_files = [f for f in os.listdir(f"segmented_audio/conversationA/00{number}")]
    for input_file in input_files:
        predicted = transcribe_Whisper_tiny(f'segmented_audio/conversationA/00{number}/{input_file}')
        match = re.search(r'(\d+\.\d+)_(\d+\.\d+)\.wav$', input_file)
        start_time = match.group(1)
        end_time = match.group(2)
        df_whisper_tiny.loc[len(df_whisper_tiny)] = {'Filename': f'D00{number}_A','Start_time': start_time,'End_time': end_time,'Whisper_tiny': predicted}

for i in range(1, 71):
    number = str(i).zfill(2)
    input_files = [f for f in os.listdir(f"segmented_audio/conversationB/00{number}")]
    for input_file in input_files:
        predicted = transcribe_Whisper_tiny(f'segmented_audio/conversationB/00{number}/{input_file}')
        match = re.search(r'(\d+\.\d+)_(\d+\.\d+)\.wav$', input_file)
        start_time = match.group(1)
        end_time = match.group(2)
        df_whisper_tiny.loc[len(df_whisper_tiny)] = {'Filename': f'D00{number}_B','Start_time': start_time,'End_time': end_time,'Whisper_tiny': predicted}

for i in range(1, 71):
    number = str(i).zfill(2)
    input_files = [f for f in os.listdir(f"segmented_audio/voice_command/00{number}")]
    for input_file in input_files:
        predicted = transcribe_Whisper_tiny(f'segmented_audio/voice_command/00{number}/{input_file}')
        match = re.search(r'(\d+\.\d+)_(\d+\.\d+)\.wav$', input_file)
        start_time = match.group(1)
        end_time = match.group(2)
        df_whisper_tiny.loc[len(df_whisper_tiny)] = {'Filename': f'P00{number}','Start_time': start_time,'End_time': end_time,'Whisper_tiny': predicted}

df_whisper_tiny.to_csv('predicted_transcription/Whisper-tiny.csv')

# EVALUATION

In [5]:
# !pip3 install jiwer
!pip3 install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.2 MB/s eta 0:00:00a 0:00:01


In [6]:
import pandas as pd
import numpy as np

df_whisper_tiny = pd.read_csv('predicted_transcription/Whisper-tiny.csv', index_col=0)
df_A_cleaned = pd.read_csv('ground_truth_cleaned/D00XX_A_ground_truth_cleaned.csv', index_col=0)
df_B_cleaned = pd.read_csv('ground_truth_cleaned/D00XX_B_ground_truth_cleaned.csv', index_col=0)
df_P_cleaned = pd.read_csv('ground_truth_cleaned/P00XX_ground_truth_cleaned.csv', index_col=0)
df_whisper_tiny = df_whisper_tiny.rename(columns={'Filename': 'file', 'Start_time': 'start_time', 'End_time': 'end_time'})
df_A_cleaned['file'] = df_A_cleaned['file'].str.extract(r'/(D\d{4}_A)', expand=False)
df_B_cleaned['file'] = df_B_cleaned['file'].str.extract(r'/(D\d{4}_B)', expand=False)
df_P_cleaned['file'] = df_P_cleaned['file'].str.extract(r'/(P\d{4})', expand=False)
df_merged = pd.merge(df_A_cleaned, df_whisper_tiny, on=['file', 'start_time', 'end_time'], how='outer')
df_merged = pd.merge(df_B_cleaned, df_merged, on=['file', 'start_time', 'end_time'], how='right', suffixes=('_x',''))
df_merged = df_merged.drop(columns=['participant_x', 'ground_truth_x', 'ground_truth_cleaned_x'])
df_merged = pd.merge(df_merged, df_P_cleaned, on=['file', 'start_time', 'end_time'], how='outer', suffixes=('','_x'))
df_merged['participant'] = df_merged['participant'].combine_first(df_merged['participant_x'])
df_merged['ground_truth'] = df_merged['ground_truth'].combine_first(df_merged['ground_truth_x'])
df_merged['ground_truth_cleaned'] = df_merged['ground_truth_cleaned'].combine_first(df_merged['ground_truth_cleaned_x'])
df_merged = df_merged.drop(columns=['participant_x', 'ground_truth_x', 'ground_truth_cleaned_x'])


In [7]:
#wordnet similarity, glove, bert, sent2vec
import jiwer
from nltk.translate.bleu_score import sentence_bleu
import nltk

# nltk.download('punkt')

WER = []
CER = []
BLEU = []
for i in range(df_merged.shape[0]):
    reference = df_merged['ground_truth_cleaned'].iloc[i]
    hypothesis = df_merged['Whisper_tiny'].iloc[i]
    if not isinstance(reference, str) or not isinstance(hypothesis, str):
        WER.append(np.nan)
        CER.append(np.nan)
        BLEU.append(np.nan)
    else:
        WER.append(jiwer.wer(reference, hypothesis))
        CER.append(jiwer.cer(reference, hypothesis))
        BLEU.append(sentence_bleu([list(reference)], list(hypothesis)))
df_merged['WER'] = WER
df_merged['CER'] = CER
df_merged['BLEU'] = BLEU

/rprojectnb/sparkpit/dlee5/.conda/envs/my_conda_env2/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/rprojectnb/sparkpit/dlee5/.conda/envs/my_conda_env2/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/rprojectnb/sparkpit/dlee5/.conda/envs/my_conda_env2/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many

In [8]:
df_merged

,file,start_time,end_time,participant,ground_truth,ground_truth_cleaned,Whisper_tiny,WER,CER,BLEU
0,D0001_A,87.970,94.850,1.0,大/r[大]家好，我是叫<姓名>，那。,大家好，我是叫<姓名>，那。,icas良好的大家好,1.0,1.000000,2.473563e-78
1,D0001_A,95.580,102.830,1.0,我是，今年是，<年龄>岁来自<居住地>，呃/i。,我是，今年是，<年龄>岁来自<居住地>，。,我是金莲是,1.0,0.857143,3.784014e-156
2,D0001_A,104.090,105.680,1.0,我是。,我是。,爸...,1.0,1.333333,0.000000e+00
3,D0001_A,106.300,119.120,1.0,资深/p的口/b[口/r/b]吃患者。,资深的口吃患者。,为什么蛮 perceive Freaky,3.0,2.500000,0.000000e+00
4,D0001_A,119.130,124.490,1.0,我是从小就有口吃。,我是从小就有口吃。,我吃 藏下就有口吃,2.0,0.555556,3.155985e-01
...,...,...,...,...,...,...,...,...,...,...
37252,P0070,3032.636,3035.926,70.0,你好，米雅，拿日元换法郎怎么换。,你好，米雅，拿日元换法郎怎么换。,"你好好米亞,拿日元換法了,這塊",1.0,0.562500,3.383742e-78
37253,P0070,3037.766,3041.436,70.0,你好，米雅，六百五十韩元是多少法郎。,你好，米雅，六百五十韩元是多少法郎。,"你好,明呀,653元是多少法狼",1.0,0.611111,2.316647e-01
37254,P0070,3043.086,3045.546,70.0,你好，米雅，添加日程。,你好，米雅，添加日程。,"你好,我们呀,天家日程",1.0,0.727273,7.746340e-155
37255,P0070,3049.256,3053.116,70.0,我觉得推翻现有数学体系不是那么不可思议了。,我觉得推翻现有数学体系不是那么不可思议了。,我覺得推翻現有數學體系不是那麼不可思議了,1.0,0.380952,2.250010e-01
